**`This code shows how to use FAISS and LangChain to build a generative QA model or chatbot, using the model: gpt-3.5-turbo-16k. `**

Dataset: CGSQuAD

Dataset format: XLSX

Two main components are required: prompt and response. The richer the response, the better the generated answer is. The "context" is used as the input column, as the model performs similarity search (using embeddings indexing)

* Dataset file name: CGSQuAD.xlsx
* Necessary columns: context
* Optional columns: question, answers
* Unecessary columns: answer_start, pq_id
* Output file name: similarity_evaluation_GPT_3_5.csv, similarity_evaluation_GPT_3_5.xlsx

#Install dependencies

In [1]:
!pip install langchain
!pip install openai
!pip install colab-env -qU
!pip install tiktoken
!pip install faiss-gpu
!pip install -U sentence-transformers
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.6 MB/s eta 0:00:00
ERROR: pip's dependency resolve

#Use an OpenAI API key

Requires an OpenAI account: https://platform.openai.com/



In [2]:
import os
os.environ["OPENAI_API_KEY"] = ""

#Test  some questions to make sure the API works

In [4]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003")

In [5]:
question = "ماهي أصعب لغة برمجة في العالم؟"
print(llm(question))



ليس هناك لغة برمجة واحدة تعتبر أصعب من الأخرى، كل لغة برمجة لها مستويات مختلفة من الصعوبة. بعض اللغات التي تعتبر مشوقة لتعلمها هي C++، Java، Go، Rust و Haskell.


In [6]:
question = "What are the top 4 resources to learn Golang in 2023?"
print(llm(question))



1. Golang Tutorials – This is a comprehensive guide to learning Golang, from the basics to advanced topics. 

2. Golang Code – This is a great resource for those looking to learn Golang by reading and writing code examples. 

3. The Go Programming Language – This is an official resource from the creators of Golang, providing tutorials, documentation, and resources for learning the language.

4. Go by Example – This is an interactive learning site with code examples to help you learn Golang quickly.


#Create an auto prompt and chain it to the model

In [7]:
from langchain import PromptTemplate
template = "What are the top {k} resources to learn {this} in 2023?"
prompt = PromptTemplate(template=template,input_variables=['k','this'])

from langchain import LLMChain
chain = LLMChain(llm=llm,prompt=prompt)

In [8]:
input = {'k':3,'this':'cooking'}
print(chain.run(input))



1. The Food Network: The Food Network has a variety of cooking shows, cooking classes, and recipe videos that can help you learn to cook. 

2. YouTube Cooking Channels: YouTube is full of helpful cooking channels filled with recipes, tutorials, and tips that can help you learn to cook. 

3. Online Cooking Courses: There are many online cooking courses available on websites such as Udemy and Skillshare. These courses offer step-by-step instructions and videos to help you learn cooking techniques.


#Import Facebook FAISS and LangChain packages

In [9]:
import os, dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain import PromptTemplate

#Use minimized unique contexts to reduce costs

In [10]:
import pandas as pd
import re
import heapq
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
data_file = "/content/CGSQuAD.xlsx"
data = pd.read_excel(data_file)
def extract_relevant_sentences(text, answer, answer_start, n_sentences=2):
    sentences = text.split('. ')#'. ' as a sentence slicer

    answer_start = answer_start#get answer location
    answer_end = answer_start + len(str(answer))

    start_index = 0#identify start and end of relevant context
    end_index = len(sentences)

    for i, sentence in enumerate(sentences):
        if sentence.lower().find(str(answer).lower()) != -1:
            counter = 1
            while i - counter >= 0 and counter <= n_sentences:#get n sentences from right and left
                start_index = i - counter
                counter += 1

            counter = 1
            while i + counter < len(sentences) and counter <= n_sentences:
                end_index = i + counter + 1
                counter += 1
            break

    new_context = '. '.join(sentences[start_index:end_index])#construct the relevant context out of the extracted sentences

    if not new_context.endswith('.'):#add a period at the last sentence
        new_context += '.'

    return new_context

data['context'] = data.apply(lambda row: extract_relevant_sentences(row['context'], row['answers'], row['answer_start']), axis=1)#for each context, call the function
data.to_csv("minimized_contexts.csv", index=False)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:
data['context'][0]#first sentence does not have right only left, thus = 3 sentences

'رسالة كلية الدراسات العليا هي العمل المخطط الهادف الى المساهمة في تنمية إمكانات جامعة الكويت لتكون مؤسسة علمية تعليمية متميزة. أهداف كلية الدراسات العليا تشمل اتاحة فرص تعليم ومواكبة التطور العالمي وصنع المثقف العصري وتناول قضايا المجتمع الكويتي. مهام كلية الدراسات العليا تشمل الموافقة على برامج الدراسات العليا ووضع الأنظمة وتحديد أسس القبول وعمل التقويم الدوري وطرح برامج اختصاصية.'

In [12]:
import pandas as pd

df=data
unique_contexts = df["context"].unique()#extract unique contexts
unique_df = pd.DataFrame({"context": unique_contexts})#new df to hold the unique contexts
print(unique_df.count())
output_file = "unique_contexts.csv"
unique_df.to_csv(output_file, index=False)

context    272
dtype: int64


#Calculate the number of tokens using NLTK and OpenAI Tiktoken

This is a method that helps in estimating billings. Detailed description of the billings of the made API requests are found here: https://platform.openai.com/account/usage

In [13]:
import pandas as pd
import nltk
import tiktoken
nltk.download('perluniprops')
nltk.download('nonbreaking_prefixes')
nltk.download('punkt')

from nltk.tokenize import word_tokenize

num_tokens_list_nltk = []
num_tokens_list_tiktoken = []
i=1
mysum=0
for context in df["context"].unique():
    print("Context: ", i)
    tokens_nltk = word_tokenize(context)#tokenize
    num_tokens_nltk = len(tokens_nltk)
    print("NLTK: ", num_tokens_nltk)
    num_tokens_list_nltk.append(num_tokens_nltk)

    #choose model first, each OpenAI model has different toenization method
    #gpt-4 Turbo, gpt-4 and gpt-3.5 Turbo have the same tokenizer
    tiktoken_encoding = tiktoken.encoding_for_model("gpt-4")
    tokens_tiktoken = tiktoken_encoding.encode(context)#tokenize
    num_tokens_tiktoken = len(tokens_tiktoken)
    mysum = mysum + num_tokens_tiktoken
    print("Tiketoken: ", num_tokens_tiktoken)
    num_tokens_list_tiktoken.append(num_tokens_tiktoken)
    i=i+1
print("Total Tiktokens: ", mysum)

[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Context:  1
NLTK:  62
Tiketoken:  262
Context:  2
NLTK:  78
Tiketoken:  331
Context:  3
NLTK:  97
Tiketoken:  416
Context:  4
NLTK:  95
Tiketoken:  412
Context:  5
NLTK:  96
Tiketoken:  417
Context:  6
NLTK:  95
Tiketoken:  407
Context:  7
NLTK:  94
Tiketoken:  395
Context:  8
NLTK:  92
Tiketoken:  370
Context:  9
NLTK:  83
Tiketoken:  319
Context:  10
NLTK:  73
Tiketoken:  261
Context:  11
NLTK:  70
Tiketoken:  250
Context:  12
NLTK:  73
Tiketoken:  259
Context:  13
NLTK:  74
Tiketoken:  270
Context:  14
NLTK:  74
Tiketoken:  275
Context:  15
NLTK:  74
Tiketoken:  278
Context:  16
NLTK:  71
Tiketoken:  270
Context:  17
NLTK:  72
Tiketoken:  284
Context:  18
NLTK:  68
Tiketoken:  265
Context:  19
NLTK:  83
Tiketoken:  341
Context:  20
NLTK:  86
Tiketoken:  345
Context:  21
NLTK:  83
Tiketoken:  344
Context:  22
NLTK:  89
Tiketoken:  379
Context:  23
NLTK:  98
Tiketoken:  425
Context:  24
NLTK:  96
Tiketoken:  420
Context:  25
NLTK:  98
Tiketoken:  436
Context:  26
NLTK:  97
Tiketoken: 

#Load dataset and index it using FAISS

In [14]:
loader = CSVLoader("/content/unique_contexts.csv")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

faissIndex = FAISS.from_documents(docs, OpenAIEmbeddings())
faissIndex.save_local("graduate_studies_docs")#saves a folder to hold index.faiss and index.pkl

#Create the QA chatbot by injecting CGSQuAD to the model

Maximum number of tokens for any generated output = max_tokens

In [15]:
dotenv.load_dotenv()
chatbot = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(
        openai_api_key=os.getenv("OPENAI_API_KEY"),
        temperature=0, model_name="gpt-3.5-turbo-16k", max_tokens=75
    ),
    chain_type="stuff",
    retriever=FAISS.load_local("graduate_studies_docs", OpenAIEmbeddings())
        .as_retriever(search_type="similarity", search_kwargs={"k":1})
)
template = """
respond as succinctly as possible. {query}?
"""
prompt = PromptTemplate(
    input_variables=["query"],
    template=template,
)

#Test and evaluate the model on CGSQuAD

Asking the same question exactly. Same generated answers indicate a stable and strong model.

In [16]:
print(chatbot.run(
    prompt.format(query=" ما هي شروط القبول؟"))
)

شروط القبول تتضمن الحصول على معدل مطلوب واجتياز مواد دراسية محددة.


In [17]:
print(chatbot.run(
    prompt.format(query=" ما هي شروط القبول؟"))
)

شروط القبول تتضمن الحصول على معدل مطلوب واجتياز مواد دراسية محددة.


Asking an unpopular/broad question

In [18]:
print(chatbot.run(
    prompt.format(query=" ما هي شروط القبول  إذا كان المتقدم من فئة المعاقين ؟"))
)

شروط القبول المشروط للماجستير إذا كان المتقدم من فئة المعاقين هي معدل عام 2.33.


Asking the same question differently

In [19]:
print(chatbot.run(
    prompt.format(query=" كم هو معدل القبول للطالب المقيد؟"))
)

معدل القبول للطالب المقيد هو 2.67 نقاط من أصل 4.00 نقاط.


In [20]:
print(chatbot.run(
    prompt.format(query=" ما هو معدل القبول المطلوب للطالب المقيد؟"))
)

معدل القبول المطلوب للطالب المقيد هو معدل عام 2.67 نقاط من أصل 4.00 نقاط.


*In here, the same question is asked, however, the model gave a different answer. The answer is not wrong, the model did not understand the question as it it not clear. The question needs to specify: is it the GPA of applying to the CGS or the required GPA to change student status from unenrolled into enrolled student. These are two different cases and the model thought this refers to the second case. The answer is correct if that is the intended question*


In [21]:
print(chatbot.run(
    prompt.format(query=" ما هو المعدل المطلوب للطالب المقيد؟"))
)

المعدل المطلوب للطالب المقيد هو 2.67 نقطة.


Asking a question that has no explicit answer within the context

In [22]:
print(chatbot.run(
    prompt.format(query=" ما الفرق بين الطالب المقيد والغير مقيد؟"))
)

الطالب المقيد هو الطالب المستوفي لجميع الشروط، بينما الطالب الغير مقيد هو الطالب الذي لم يستوفِ شرط المعدل المطلوب لل


Asking questions that might appear similar but are not

In [23]:
print(chatbot.run(
    prompt.format(query=" ما هو الطالب المقيد؟"))
)

الطالب المقيد هو الطالب المستوفي لجميع الشروط والمعايير المطلوبة للقبول في الدراسات العليا.


In [24]:
print(chatbot.run(
    prompt.format(query=" الطالب الغير مقيد هو؟"))
)

الطالب الغير مقيد هو الطالب الذي لم يستوفي شرط المعدل المطلوب للقبول في الدراسات العليا.


In [25]:
print(chatbot.run(
    prompt.format(query=" ما هو الطالب غير المقيد؟"))
)

الطالب غير المقيد هو الطالب الذي لم يستوفِ شرط المعدل المطلوب للقبول في الدراسات العليا.


In [26]:
print(chatbot.run(
    prompt.format(query=" ما هو معدل القبول المطلوب من الطالب غير المقيد؟"))
)

معدل القبول المطلوب من الطالب غير المقيد هو معدل عام 2.50 نقاط من أصل 4.00 نقاط.


In [27]:
print(chatbot.run(
    prompt.format(query="what are the offered programs? "))
)

The offered programs can be found on the website of the College of Graduate Studies: http://www.kuniv.edu.kw/COGS/index.htm


In [28]:
print(chatbot.run(
    prompt.format(query="where to apply for IELTS?"))
)

You can apply for IELTS at the IDP institute in Kuwait.


In [29]:
print(chatbot.run(
    prompt.format(query="how much is the allowance for the external examiner?"))
)

The allowance for the external examiner is 500 Kuwaiti dinars.


In [30]:
print(chatbot.run(
    prompt.format(query="what is an enrolled student?"))
)

An enrolled student is a student who has met all the requirements and is officially registered in a program of study.


In [31]:
print(chatbot.run(
    prompt.format(query="what is an unenrolled student?"))
)

An unenrolled student is a student who has not met all the requirements for enrollment or has not completed the necessary steps to officially register for a program or course.


Test the model on n questions. Use manual evaluation & semantic similarity check. Sometimes the API call disconnects

In [32]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import torch

def calculate_similarity(sentence1, sentence2):
    # Ensure sentence1 and sentence2 are treated as strings
    sentence1 = str(sentence1)
    sentence2 = str(sentence2)

    # Calculate embeddings
    model = SentenceTransformer("symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli")
    embedding1 = model.encode(sentence1, convert_to_tensor=True)
    embedding2 = model.encode(sentence2, convert_to_tensor=True)

    # Calculate cosine similarity between embeddings
    similarity_score = torch.nn.functional.cosine_similarity(embedding1.unsqueeze(0), embedding2.unsqueeze(0)).item()
    return similarity_score

# Your dataframe and other code

dfx = df  # Assuming df is defined somewhere

questions = dfx["question"]
answers = dfx["answers"]

random_questions = questions.sample(n=1500, random_state=42)

random_question = []
generated_responses = []
corresponding_responses = []
similarity_scores = []

pbar = tqdm(total=len(random_questions), desc="Calculating Similarity")

for question in random_questions:
    generated_response = chatbot.run(prompt.format(query=question))
    random_question.append(question)
    generated_responses.append(generated_response)

    # Use boolean indexing to find the corresponding response
    corresponding_response = answers[questions == question].values[0]
    corresponding_responses.append(corresponding_response)

    similarity = calculate_similarity(generated_response, corresponding_response)
    similarity_scores.append(similarity)

    pbar.update(1)
pbar.close()

mean_similarity_score = np.mean(similarity_scores)

print("Mean Similarity Score:", mean_similarity_score)

evaluation_df = pd.DataFrame({
    "random_question": random_question,
    "generated_response": generated_responses,
    "corresponding_response": corresponding_responses,
    "similarity_score": similarity_scores
})


Calculating Similarity:   0%|          | 0/1500 [00:00<?, ?it/s]

(…)5072583e5fbba7a9c69bd8606/.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

(…)e5fbba7a9c69bd8606/1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(…)8bdff365072583e5fbba7a9c69bd8606/LICENSE:   0%|          | 0.00/22.3k [00:00<?, ?B/s]

(…)dff365072583e5fbba7a9c69bd8606/README.md:   0%|          | 0.00/2.73k [00:00<?, ?B/s]

...

(…)5072583e5fbba7a9c69bd8606/tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

(…)e5fbba7a9c69bd8606/tokenizer_config.json:   0%|          | 0.00/356 [00:00<?, ?B/s]

(…)365072583e5fbba7a9c69bd8606/modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

Calculating Similarity: 100%|██████████| 1500/1500 [2:33:17<00:00,  6.13s/it]

Mean Similarity Score: 0.47364888205938044


In [33]:
evaluation_df.to_csv("similarity_evaluation_GPT_3_5.csv", index=False)

In [34]:
evaluation_df.to_csv("similarity_evaluation_GPT_3_5.xlsx", index=False)

In [35]:
evaluation_df

,random_question,generated_response,corresponding_response,similarity_score
0,كيف تتألف لجنة البرنامج؟,لجنة البرنامج تتألف من ثلاثة أعضاء على الأقل، ...,عن طريق عميد كلية الدراسات العليا,0.436209
1,كم عدد المشاركات العلمية التي يستطيع الطالب حض...,الطالب يستطيع حضور مهمة علمية واحدة فقط.,مهمة واحدة فقط خلال مدة دراسته,0.607484
2,هل يشترط اعداد الرسالة للحصول على دبلوم دراسات...,لا، لا يشترط اعداد الرسالة للحصول على دبلوم در...,ولا يشترط اجتياز الامتحان الشامل او اعداد أطروحة,0.210208
3,ما هي قيمة بدل السفر اليومي لتكاليف الرحلات ال...,قيمة بدل السفر اليومي لتكاليف الرحلات العلمية ...,30 دينار كويتي,0.674283
4,ماذا يحصل اذا سقط الطالب في اختبار اطروحة الدك...,اذا سقط الطالب في اختبار اطروحة الدكتوراة، يمن...,يمنح فرصة أخرى,0.178025
...,...,...,...,...
1495,هل يشترط حضور مختبر اختبار بحث الدكتوراة اثناء...,نعم، يشترط حضور ممتحن امتحان رسالة الدكتوراة أ...,نعم يشترط حضور ممتحن امتحان رسالة الدكتوراة,0.916464
1496,ما هي شروط القبول المشروط للماجستير إذا كان ال...,شروط القبول المشروط للماجستير إذا كان المتقدم ...,معدل عام 2.33,0.683547
1497,كم قيمة مكافأة لجنة الاختبار الشفهي لكلية الحقوق؟,قيمة مكافأة لجنة الاختبار الشفهي لكلية الحقوق ...,100 دينار كويتي لكل عضو,0.889421
1498,ما هو المعدل الادنى للتخرج من دبلوم الدراسات ا...,المعدل الأدنى للتخرج من الدبلوم العالي هو 2.67...,هو 2.67,0.576328
